# <center> Figures for Dynamic Star Alleles paper</center>
--- 

## 1. Setting up environment

 Load rpy2 package to use both R and python within the same notebook.

In [ ]:
%load_ext rpy2.ipython


Import python packages and R libraries.

In [ ]:
import glob
from itertools import product
import pandas as pd
import plotly.graph_objects as go
import os
import re

import pypgx

from utils import *

In [ ]:
%%R
library("ggalluvial")
library("lubridate")
library("cowplot")
library("extrafont")
library("ggsci")
library("devtools")
library("ggradar")
library("grid")
library("ggthemes")
library("scales")
library("dplyr")
library("ggplot2")
library("patchwork")
library("tidyverse")

Define global variables.

In [ ]:
CONSOLIDATED_GETRM_CONSENSUS_FILE = "path/to/consolidated_getrm_consensus"
UPDATED_CALLS = "/path/to/updated_calls"

## 2. Creating Figures.

### 2.1 Preparing datasets

In [ ]:
all_updated_calls = (
    pd.read_excel(UPDATED_CALLS,
                        sheet_name = "Updated Calls",
                        header = 0)
                        .filter(["Sample", "Gene", "GeT-RM", "Updated Call"])
                        .reset_index()
)

getrm_consolidated_calls = (
    pd.read_excel(CONSOLIDATED_GETRM_CONSENSUS_FILE, 
                  sheet_name = "PGx_Panel",
                  header = 1)
)

getrm_columns = getrm_consolidated_calls.columns.to_list()
getrm_columns.remove("VKORC1 rs9923231 ONLY")


getrm_consolidated_calls = getrm_consolidated_calls[getrm_consolidated_calls["ENA-id"].notnull()].reset_index(drop = True)

getrm_consolidated_calls = getrm_consolidated_calls.filter(getrm_columns)
getrm_consolidated_calls = getrm_consolidated_calls.rename(columns = {"Coriell #" : "Sample"})
# keep only the rows that have a value in th ENA-id

## remove rows that have no value in Updated Call column

all_calls_counts = all_updated_calls["Gene"].value_counts().reset_index()
all_calls_counts["Not-updated"] = 70 - all_calls_counts["count"]
all_calls_counts.columns = ["Gene", "Updated", "Not-updated"]


all_calls_counts_tidy = pd.melt(all_calls_counts, id_vars = "Gene", value_vars = ["Updated", "Not-updated"], value_name = "Count")
all_calls_counts_tidy.columns = ["Gene", "Update_status", "Count"]

# number of samples that are updated
print(all_updated_calls["Sample"].nunique())



### 2.2 Define helper functions for classification of calls

In [ ]:
# Helper functions for classification of calls
def convert_diplotype_to_alleles(diplotype):
    print(diplotype)
    alleles = diplotype.split("/")
    # in the case of no diplotype in getrm:
    x = [i for i in alleles if i != ""]
    if len(x) == 0:
        return None
    return [allele.replace("*", "") for allele in alleles]

def convert_legacy_to_new(legacy_allele_map: dict[str], alleles: list[str], legacy_allele_list: list[str]):
    if alleles is None:
        return None
    if legacy_allele_list:
        return [legacy_allele_map[allele].replace(" ","") if allele in legacy_allele_map.keys() else allele for allele in alleles]
    else:
        return alleles

def process_consensus(input_list):
    consensus_1 = []  # Will hold items without brackets
    tentative = []  # Will hold numbers that were in brackets
    
    for item in input_list:
        if "(" in item or ")" in item:
            # Remove bracket and add to consensus_2
            alleles = item.replace('(', '')
            alleles = alleles.replace(')', '')
            alleles = alleles.replace(" ", "")
            alleles = alleles.split(",")
            # print(alleles)
            tentative.extend(alleles)
        else:
            consensus_1.append(item)
            
    return consensus_1, tentative

def classify_calls_test(call: list[str], consensus: list[str], gene: str, 
                  legacy_alleles_list: dict[list[str]] = legacy_alleles_list, 
                  legacy_alleles_map: dict[dict[str]] = legacy_alleles_map):
    """
    Classify allele calls against consensus alleles, handling legacy alleles and phasing uncertainty.
    
    Args:
        call: List of alleles from the update to classify
        consensus: List of alleles from the GeT-RM consensus
        gene: Gene name
        legacy_alleles_list: Dictionary mapping genes to their legacy alleles
        legacy_alleles_map: Dictionary mapping genes to their legacy->new allele mappings
    
    Returns:
        Tuple of (legacy, discordant, confirmed, concordant) counts
    """
    if not consensus:
        return 0, len(call), 0, 0

    # Initialize variables
    is_tentative_in_consensus = any('(' in item or ')' in item for item in consensus)
    consensus_str = '/'.join(consensus)
    consensus_tentative = []
    legacy = 0
    discordant = 0
    confirmed = 0
    concordant = 0

    # Process tentative consensus if needed
    if is_tentative_in_consensus:
        consensus, consensus_tentative = process_consensus(consensus)

    # Handle phase uncertainty cases first (or/;)
    if ' or ' in consensus_str or ';' in consensus_str:
        possible_diplotypes = consensus_str.split(' or ') if ' or ' in consensus_str else consensus_str.split(';')
        
        if gene in legacy_alleles_list.keys():
            legacy_alleles_list_gene = legacy_alleles_list[gene] or None
            legacy_alleles_map_gene = legacy_alleles_map[gene] or None

            if gene == "CYP1A2":
                call = convert_legacy_to_new(legacy_alleles_map_gene, call, legacy_alleles_list_gene)

            # Try complete diplotype matches first
            for diplotype in possible_diplotypes:
                original = convert_diplotype_to_alleles(diplotype)
                converted = convert_legacy_to_new(legacy_alleles_map_gene, original, legacy_alleles_list_gene)
                
                if sorted(call) == sorted(converted):
                    # Found exact match - count legacy alleles
                    legacy_count = sum(1 for allele in original if allele in legacy_alleles_list_gene)
                    return legacy_count, 0, 0, len(call) - legacy_count
            
            # No complete match - check individual alleles
            working_call = call.copy()
            for allele in working_call:
                found_legacy = False
                # First check if it's a legacy allele itself (for CYP1A1)
                if gene == "CYP1A1" and allele in legacy_alleles_list_gene:
                    print(allele)
                    mapped_allele = legacy_alleles_map_gene[allele]
                    if mapped_allele in consensus:
                        legacy += 1
                        found_legacy = True
                # Then check normal legacy matching
                if not found_legacy:
                    for orig_allele in legacy_alleles_list_gene:
                        if legacy_alleles_map_gene[orig_allele].replace(" ", "") == allele:
                            legacy += 1
                            found_legacy = True
                            break
                if not found_legacy:
                    discordant += 1
            return legacy, discordant, 0, 0
        else:
            # Non-legacy gene with phase uncertainty
            for diplotype in possible_diplotypes:
                diplotype_alleles = convert_diplotype_to_alleles(diplotype)
                if sorted(call) == sorted(diplotype_alleles):
                    return 0, 0, 0, len(call)  # All concordant if exact match
            return 0, len(call), 0, 0  # No match - all discordant

    # Handle non-phase uncertainty cases
    if gene in legacy_alleles_list.keys():
        legacy_alleles_list_gene = legacy_alleles_list[gene] or None
        legacy_alleles_map_gene = legacy_alleles_map[gene] or None
        original_consensus = consensus.copy()

        if gene == "CYP1A2":
            call = convert_legacy_to_new(legacy_alleles_map_gene, call, legacy_alleles_list_gene)

        consensus_converted = convert_legacy_to_new(legacy_alleles_map_gene, consensus, legacy_alleles_list_gene)
        consensus_tentative = convert_legacy_to_new(legacy_alleles_map_gene, consensus_tentative, legacy_alleles_list_gene)

        working_consensus_converted = consensus_converted.copy()
        working_consensus_tentative = consensus_tentative.copy()
        working_original = original_consensus.copy()
        working_call = call.copy()

        for allele in working_call:
            found_match = False
            # Check for CYP1A1 legacy alleles first
            if gene == "CYP1A1" and allele in legacy_alleles_list_gene:
                mapped_allele = legacy_alleles_map_gene[allele]
                if mapped_allele in working_consensus_converted:
                    legacy += 1
                    working_consensus_converted.remove(mapped_allele)
                    found_match = True
            
            if not found_match:
                if allele in working_consensus_converted:
                    pos = working_consensus_converted.index(allele)
                    original_allele = working_original[pos]
                    if original_allele in legacy_alleles_list_gene:
                        legacy += 1
                    else:
                        concordant += 1
                    working_consensus_converted.remove(allele)
                    working_original.pop(pos)
                elif allele in working_consensus_tentative:
                    confirmed += 1
                    working_consensus_tentative.remove(allele)
                else:
                    discordant += 1

    else:
        # Handle normal non-legacy cases
        working_consensus = consensus.copy()
        working_consensus_tentative = consensus_tentative.copy()

        for allele in call:
            if allele in working_consensus:
                concordant += 1
                working_consensus.remove(allele)
            elif allele not in working_consensus and allele not in working_consensus_tentative:
                discordant += 1
            elif allele in working_consensus_tentative:
                confirmed += 1
                working_consensus_tentative.remove(allele)

    return legacy, discordant, confirmed, concordant

## 2.3 Classify alleles of reevaluated dataset

In [ ]:
### Code to count legacy, discordant, confirmed and concordant calls
cyp1a1_legacy_alleles = ["2A", "2B", "2C"]
cyp1a1_legacy_allele_map = {"2A": "2", "2B": "2", "2C": "2"}


slco1b1_legacy_alleles= ["1A", "1B", "17", "18", "20", "21", "22", "35"]
slco1b1_legacy_allele_map = {"1A": "1", "1B": "37", "17": "15", "18": "14", "20": "20", "21": "20", "22": "19", "35": "20"}

cyp1a2_legacy_alleles = ["1A", "1B", "1F", "1L", "1M", "1N", "1P", "1Q", "1R", "1S", "1T", "1U", "S298R"]
cyp1a2_legacy_allele_map = {"1A": "1", "1B": "1", "1F": "30", "1L": "30", "1M": "30", "1N": "30", "1P": "30", "1Q": "30", "1R": "30", "1S": "1", "1T": "1", "1U": "1", "S298R" : "1"}

cyp2a6_legacy_alleles = ["4D"]
cyp2a6_legacy_allele_map = {"4D": "47"}

cyp2c19_legacy_alleles = ["27", "1A"]
cyp2c19_legacy_allele_map = {"27": "1", "1A": "38"}

cyp2d6_legacy_alleles = ["57", "14A"]
cyp2d6_legacy_allele_map = {"57": "36", "14A": "114"}

cyp3a4_legacy_alleles = ["1A", "1B"]
cyp3a4_legacy_map = {"1A": "1", "1B": "1"}

cyp3a5_legacy_alleles = ["2","3A", "3B", "3D", "3F", "3G", "3J", "3K", "3L", "4", "5", "10", "11"]
cyp3a5_legacy_map = {
    "2": "3",
    "3A": "3",
    "3B": "3",
    "3D": "3",
    "3F": "3",
    "3G": "3",
    "3J": "3",
    "3K": "3",
    "3L": "3",
    "4": "3",
    "5": "3",
    "10": "3",
    "11": "3"
}

nat2_legacy_alleles = ["12A", "12B", "12C", "12I", "11A", "13A", "20", "5C", "5B", "5F", "5W", "5BB", "6B", "6A", "6D", "6E", "6L", "7A", "7B", "14A", "14B", "14D", "5D", "5A", "14F", "14C", "5E", "5L", "5O", "5N", "6C", "6M", "6P", "6K", "6H", "6O", "7C", "12E", "12F", "12G", "12H", "12J", "14E", "14G", "14I", "14H", "12D", "12N", "6G"]
nat2_legacy_map = {
    "12A" : "1",
    "12B" : "1",
    "12C" : "1",
    "12I" : "1",
    "11A" : "4",
    "13A" : "4",
    "20" : "4",
    "5C" : "5",
    "5B" : "5",
    "5F" : "5",
    "5W" : "5",
    "5BB" : "5",
    "6B" : "6",
    "6A" : "6",
    "6D" : "6",
    "6E" : "6",
    "6L" : "6",
    "7A" : "7",
    "7B" : "7",
    "14A" : "14",
    "14B" : "14",
    "14D" : "15",
    "5D" : "16",
    "5A" : "16",
    "14F" : "29",
    "14C" : "29",
    "5E" : "30",
    "5L" : "31",
    "5O" : "32",
    "5N" : "33",
    "6C" : "34",
    "6M" : "35",
    "6P" : "36",
    "6K" : "37",
    "6H" : "38",
    "6O" : "39",
    "7C" : "40",
    "12E" : "41",
    "12F" : "42",
    "12G" : "43",
    "12H" : "44",
    "12J" : "45",
    "14E" : "46",
    "14G" : "46",
    "14I" : "46",
    "14H" : "47",
    "12D" : "48",
    "12N" : "51",
    "6G" : "52"
}

legacy_alleles_list = {
    "SLCO1B1": slco1b1_legacy_alleles,
    "CYP1A2": cyp1a2_legacy_alleles,
    "CYP2A6": cyp2a6_legacy_alleles,
    "CYP3A4": cyp3a4_legacy_alleles,
    "CYP1A1": cyp1a1_legacy_alleles
}
legacy_alleles_map = {
    "SLCO1B1": slco1b1_legacy_allele_map,
    "CYP1A2": cyp1a2_legacy_allele_map,
    "CYP2A6": cyp2a6_legacy_allele_map,
    "CYP3A4": cyp3a4_legacy_map,
    "CYP1A1": cyp1a1_legacy_allele_map
}


genes_with_legacy_alleles = ["CYP1A1", "SLCO1B1", "CYP1A2", "CYP2A6", "CYP3A4"]

updated_calls_classified = all_updated_calls.copy()

updated_calls_classified["legacy"] = updated_calls_classified.apply(lambda x: classify_calls_test(convert_diplotype_to_alleles(x["Updated Call"]), convert_diplotype_to_alleles(x["GeT-RM"]), x["Gene"])[0], axis = 1)
updated_calls_classified["discordant"] = updated_calls_classified.apply(lambda x: classify_calls_test(convert_diplotype_to_alleles(x["Updated Call"]), convert_diplotype_to_alleles(x["GeT-RM"]), x["Gene"])[1], axis = 1)
updated_calls_classified["confirmed"] = updated_calls_classified.apply(lambda x: classify_calls_test(convert_diplotype_to_alleles(x["Updated Call"]), convert_diplotype_to_alleles(x["GeT-RM"]), x["Gene"])[2], axis = 1)
updated_calls_classified["concordant"] = updated_calls_classified.apply(lambda x: classify_calls_test(convert_diplotype_to_alleles(x["Updated Call"]), convert_diplotype_to_alleles(x["GeT-RM"]), x["Gene"])[3], axis = 1)


updated_calls_classified_by_gene = updated_calls_classified.groupby("Gene").agg({
    "Gene" : lambda x: len(x)*2,
    "legacy": "sum",
    "discordant": "sum",
    "confirmed": "sum",
    "concordant": "sum"
})

updated_calls_classified_by_gene = updated_calls_classified_by_gene.rename(columns = {"Gene": "updates"}).reset_index()

updated_calls_classified_by_gene["original"] = 140 - updated_calls_classified_by_gene["legacy"] - updated_calls_classified_by_gene["discordant"] - updated_calls_classified_by_gene["confirmed"] - updated_calls_classified_by_gene["concordant"]	
updated_calls_classified_by_gene = updated_calls_classified_by_gene.drop(columns = ["updates"], axis = 1)



updated_calls_classified_tidy = pd.melt(updated_calls_classified_by_gene, id_vars = "Gene", value_vars = ["legacy", "discordant", "confirmed", "concordant", "original"], value_name = "Count")



### Figure 2: Classification of alleles of reevaluated dataset

In [ ]:
%%R -i updated_calls_classified_tidy -h 800 -w 1200
# descending_order <- c("SLCO1B1", "CYP2A6", "CYP4F2", "CYP1A1", "NAT2","NAT1", "GSTT1", "CYP2B6", "CYP2C19", "GSTP1", "DPYD", "CYP2C9", "CYP2D6", "UGT1A1", "CYP2C8", "GSTM1","CYP1A2","TPMT")
# ascending_order <- rev(descending_order)

colors <- rev(c("#FF6961", ggsci::pal_igv()(8)[2:4], "#ADD8E6"))

gene_order <- c("SLCO1B1", "CYP2A6", "CYP4F2", "CYP1A1", "NAT2","NAT1", "GSTT1", "CYP2B6","GSTP1", "CYP2C19", "CYP1A2", "CYP2D6", "UGT1A1", "CYP3A4","GSTM1","TPMT")

# only keep the genes that are in the gene_order list
updated_calls_classified_tidy <- updated_calls_classified_tidy[updated_calls_classified_tidy$Gene %in% gene_order,]

updated_calls_classified_tidy$Gene <- factor(updated_calls_classified_tidy$Gene, levels = gene_order)


updated_calls_classified_tidy$variable <- factor(
  updated_calls_classified_tidy$variable, 
  levels = rev(c("discordant", "legacy", "concordant", "confirmed", "original"))
)

updated_calls_classified_tidy <- updated_calls_classified_tidy %>%
  group_by(Gene) %>%
  arrange(Gene, desc(variable)) %>%
  mutate(label_y = cumsum(Count) - 0.5 * Count)

updated_calls_plot <- ggplot(updated_calls_classified_tidy, aes(x = Gene, y = Count, fill = variable)) +
    geom_bar(stat = "identity", position = "stack", color = "black") +
    geom_text(aes(y = label_y, label = Count), 
              color = "black", size = 2, fontface = "bold") +
    scale_fill_manual(values = colors) + 
    theme_minimal(base_size = 7) +
    theme(
        axis.text.x = element_text(angle = 45, hjust = 1, vjust = 1, size = 7),
        axis.text.y = element_text(size = 7, hjust = 1, vjust = 0.5),
        axis.title = element_text(size = 7),
        legend.title = element_text(size = 7),
        legend.text = element_text(size = 7),
        legend.key.size = unit(0.4, "cm"),
        panel.grid = element_blank(),
        legend.position = "right",
        plot.title = element_text(size = 8, face = "bold", hjust = 0.5)
    ) +
    scale_y_continuous(breaks = c(0, 70, 140)) + 
    labs(x = "Gene", y = "Number of star alleles", fill = "Allele Classification")


## 2.4 Predict phenotypes of updated calls

In [ ]:

genes_with_phenotype_data = ["CYP2B6", "CYP2C9", "CYP2C19", "CYP2D6", "SLCO1B1", "TPMT", "UGT1A1"]

getrm_samples = [
    "NA11839", "NA18519", "NA20509", "NA18524", "NA19176", "NA07357", "NA12717", "NA19109", 
    "NA20296", "NA19174", "NA19178", "HG00436", "NA18509", "NA18518", "NA18484", "NA19147", 
    "NA18565", "NA18959", "NA18966", "NA18540", "HG00589", "NA19207", "NA19143", "NA18855", 
    "NA07348", "NA10854", "NA12873", "NA18992", "NA18980", "NA07019", "NA11993", "NA10831", 
    "NA19239", "NA12003", "NA18564", "NA12156", "NA07029", "NA10851", "NA12813", "NA18973", 
    "HG00276", "HG01190", "NA06991", "NA18942", "NA19789", "NA10847", "NA12006", "NA07055", 
    "NA18526", "NA07056", "NA12145", "NA18868", "NA19095", "NA19122", "NA19003", "NA19226", 
    "NA18544", "NA18952", "NA11832", "NA18861", "NA19908", "NA19819", "NA19920", "NA19007", 
    "NA19213", "NA18552", "NA18617", "NA07000", "NA19917", "NA21781"
]
def convert_diplotype_to_alleles_keep_star(diplotype):
    alleles = diplotype.split("/")
    # in the case of no diplotype in getrm:
    x = [i for i in alleles if i != ""]
    if len(x) == 0:
        return None
    return [allele for allele in alleles]

def predict_phenotypes(gene: str, diplotype: str, tentative: bool = False) -> str:
    diplotype = remove_brackets(diplotype)
    if gene not in genes_with_phenotype_data:
        return "No phenotype data available"
    if "or" in diplotype:
        alleles = diplotype.split("or")
        alleles = [allele.replace(" ", "") for allele in alleles]

    if diplotype.split("/")[1] == "" or diplotype.split("/")[0] == "":
        if tentative:
            return "Tentative Phenotype"
    alleles = convert_diplotype_to_alleles_keep_star(diplotype)
    # put * in front of alleles
    # alleles = ["*" + allele for allele in alleles]
    if gene in genes_with_legacy_alleles:
        alleles = convert_legacy_to_new(legacy_alleles_map[gene], alleles, legacy_alleles_list[gene])
    if gene in ["CYP1A1", "CYP2A6", "CYP2C8", "CYP4F2"]:
        if alleles[0] == "*1":
            if alleles[1] == "*1": 
                return ["Normal Metabolizer"]
    return pypgx.predict_phenotype(gene, alleles[0], alleles[1])



def remove_brackets(diplotype: str) -> str:
    # remove everything between brackets and the brackets itself
    diplotype = re.sub(r'\([^)]*\)', '', diplotype)
    diplotype = diplotype.replace(" ", "")
    return diplotype

def tentative_call(diplotype: str) -> bool:
    ### call is tentative if there are brackets in the diplotype and the diplotype is not "complete", eg. not all alleles are present
    if "(" in diplotype or ")" in diplotype:
        diplotype = remove_brackets(diplotype)
        diplotype = diplotype.replace("*", "")
        if diplotype.split("/")[0] != "" and diplotype.split("/")[1] != "":
            # print("this diplotype is not a tentative call", diplotype, "False")
            return False
        # print("this diplotype is a tentative call", diplotype, "True")
        return True
    # print("this diplotype is not a tentative call", diplotype, "False")
    return False

def allele_list_to_diplotype(alleles: list[str]) -> str:
    return f"*{alleles[0]}/*{alleles[1]}"

def use_wt_allele_for_tentatives(diplotype: str) -> str:
    diplotype = remove_brackets(diplotype)
    allele1 = diplotype.replace("/", "")
    wt_allele = "*1"
    return f"{wt_allele}/{allele1}"

for gene in getrm_consolidated_calls.columns[1::]:
    getrm_consolidated_calls[gene] = getrm_consolidated_calls[gene].apply(lambda x: str(x))
    getrm_consolidated_calls[gene] = getrm_consolidated_calls[gene].apply(remove_brackets)
    ### if brackets --> tentative column = TRUE


all_updated_calls["tentative"] = all_updated_calls.apply(
    lambda row: tentative_call(row["GeT-RM"]),
    axis = 1
)

all_updated_calls["GeT-RM"] = all_updated_calls.apply(
    lambda row: convert_legacy_to_new(legacy_alleles_map[row["Gene"]], convert_diplotype_to_alleles(row["GeT-RM"]), legacy_alleles_list[row["Gene"]]) if row["Gene"] == "SLCO1B1" else row["GeT-RM"],
    axis = 1
)

all_updated_calls["GeT-RM"] = all_updated_calls.apply(
    lambda row: allele_list_to_diplotype(row["GeT-RM"]) if row["Gene"] == "SLCO1B1" else row["GeT-RM"],
    axis = 1
)

all_updated_calls["GeT-RM"] = all_updated_calls.apply(
    lambda row: use_wt_allele_for_tentatives(row["GeT-RM"]) if row["tentative"] else row["GeT-RM"],
    axis = 1
)



all_updated_calls["consensus_phenotype"] = all_updated_calls.apply(
    lambda row: predict_phenotypes(row["Gene"], row["GeT-RM"], row["tentative"]), 
    axis=1
)
all_updated_calls["updated_phenotype"] = all_updated_calls.apply(
    lambda row: predict_phenotypes(row["Gene"], row["Updated Call"] ), 
    axis=1)

# all_updated_calls

all_updated_calls = all_updated_calls[all_updated_calls["Gene"].isin(genes_with_phenotype_data)]
updated_calls_with_phenotype_data = all_updated_calls.copy()

# keep only rows where the updated phenotype is different from the consensus phenotype
updated_calls_with_phenotype_data = updated_calls_with_phenotype_data[updated_calls_with_phenotype_data["consensus_phenotype"] != updated_calls_with_phenotype_data["updated_phenotype"]]
print(updated_calls_with_phenotype_data)

# remove UGT1A1 because not actual change in phenotype
updated_calls_with_phenotype_data = updated_calls_with_phenotype_data[updated_calls_with_phenotype_data["Gene"] != "UGT1A1"]

# calculate the number of samples with phenotype changes per gene
updated_calls_with_phenotype_data_by_gene = updated_calls_with_phenotype_data.groupby("Gene").agg({
    "Gene": "count"
})
updated_calls_with_phenotype_data_by_gene["percentage"] = updated_calls_with_phenotype_data_by_gene["Gene"] / 70 *100

# Remove UGT1A1 and TPMT from the dataframe
updated_calls_with_phenotype_data = updated_calls_with_phenotype_data[updated_calls_with_phenotype_data["Gene"] != "UGT1A1"]
updated_calls_with_phenotype_data = updated_calls_with_phenotype_data[updated_calls_with_phenotype_data["Gene"] != "TPMT"]



### Supplementary figure 1

In [ ]:
%%R -i updated_calls_with_phenotype_data

library(ggplot2)
library(dplyr)
library(tidyr)

# Create the data frame
data <- updated_calls_with_phenotype_data

# Create a complete matrix of all Gene-Sample combinations
all_samples <- unique(data$Sample)
all_genes <- unique(data$Gene)
complete_matrix <- expand.grid(Sample = all_samples, Gene = all_genes)

# Merge with actual data to identify changes vs no changes
data_with_changes <- complete_matrix %>%
  left_join(data, by = c("Sample", "Gene")) %>%
  mutate(has_change = !is.na(consensus_phenotype))

# Create color mapping for phenotypes with more distinct colors and combined normal categories
phenotype_colors <- c(
  "Normal Metabolizer/Normal Function" = "#90EE90",  # Light green
  "Intermediate Metabolizer" = "#FFA500",            # Orange
  "Poor Metabolizer" = "#FF4444",                    # Bright red
  "Poor Function" = "#CC0000",                       # Dark red
  "Rapid Metabolizer" = "#4169E1",                   # Royal blue
  "Decreased Function" = "#800080",                  # Purple
  "Increased Function" = "#00CED1",                  # Dark turquoise
  "Indeterminate" = "#808080",                       # Gray
  "Unchanged" = "#E8E8E8"                           # Light gray for unchanged phenotypes
)

# Modify the data to use the combined category name
data_with_changes <- data_with_changes %>%
  mutate(
    consensus_phenotype = ifelse(consensus_phenotype %in% c("Normal Metabolizer", "Normal Function"), 
                                "Normal Metabolizer/Normal Function", 
                                consensus_phenotype),
    updated_phenotype = ifelse(updated_phenotype %in% c("Normal Metabolizer", "Normal Function"), 
                              "Normal Metabolizer/Normal Function", 
                              updated_phenotype)
  )
# Create rectangles for unchanged phenotypes
unchanged_data <- data_with_changes %>%
  filter(!has_change) %>%
  mutate(
    x_pos = as.numeric(factor(Sample, levels = all_samples)),
    y_pos = as.numeric(factor(Gene, levels = rev(all_genes)))
  )

# Prepare data for triangles (only for changed phenotypes)
create_triangle_data <- function(data, type = "lower") {
  result <- data %>%
    filter(has_change) %>%
    mutate(
      x_pos = as.numeric(factor(Sample, levels = all_samples)),
      y_pos = as.numeric(factor(Gene, levels = rev(all_genes)))
    ) %>%
    group_by(Sample, Gene, consensus_phenotype, updated_phenotype, x_pos, y_pos) %>%
    do({
      if(type == "lower") {
        data.frame(
          x = c(.$x_pos - 0.5, .$x_pos + 0.5, .$x_pos - 0.5),
          y = c(.$y_pos - 0.5, .$y_pos - 0.5, .$y_pos + 0.5),
          phenotype = .$consensus_phenotype
        )
      } else {
        data.frame(
          x = c(.$x_pos + 0.5, .$x_pos + 0.5, .$x_pos - 0.5),
          y = c(.$y_pos - 0.5, .$y_pos + 0.5, .$y_pos + 0.5),
          phenotype = .$updated_phenotype
        )
      }
    }) %>%
    ungroup()
  
  return(result)
}

lower_triangles <- create_triangle_data(data_with_changes, "lower")
upper_triangles <- create_triangle_data(data_with_changes, "upper")

# Create the plot
metabolizer_classes <- ggplot() +
  # Add rectangles for unchanged phenotypes
  geom_rect(data = unchanged_data,
            aes(xmin = x_pos - 0.5, xmax = x_pos + 0.5,
                ymin = y_pos - 0.5, ymax = y_pos + 0.5),
            fill = phenotype_colors["Unchanged"],
            alpha = 0.3) +
  # Add lower triangles for consensus phenotype
  geom_polygon(data = lower_triangles,
               aes(x = x, y = y, group = interaction(Sample, Gene),
                   fill = phenotype),
               alpha = 0.8) +
  # Add upper triangles for updated phenotype
  geom_polygon(data = upper_triangles,
               aes(x = x, y = y, group = interaction(Sample, Gene),
                   fill = phenotype),
               alpha = 0.8) +
  # Customize the appearance
  scale_fill_manual(values = phenotype_colors) +
  scale_x_continuous(breaks = 1:length(all_samples),
                    labels = all_samples,
                    expand = c(0, 0)) +
  scale_y_discrete(limits = rev(all_genes)) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1),
        panel.grid = element_blank(),
        legend.position = "bottom") +
  labs(title = "Changes in Metabolizer class per Gene",
       x = "",
       y = "",
       fill = "Phenotype") +
  coord_fixed()

metabolizer_classes

